In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import logging
import networkx as nx
import os
import h5py
import matplotlib.pyplot as plt
import torch.optim as optim
import time
import math
import os, sys
import seaborn as sns
import warnings
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
# from MaxMin_Norm import MinMaxNormalization
from torch.utils.data import TensorDataset, DataLoader
sys.path.append('../data_process/')
warnings.filterwarnings("ignore")
import pickle

In [2]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=False,batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU(inplace = True)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        self.relu = nn.ReLU(inplace = True)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
class h_sigmoid(nn.Module):
    def __init__(self, inplace=True):
        super(h_sigmoid, self).__init__()
        self.relu = nn.ReLU6(inplace=inplace)
    def forward(self, x):
        return self.relu(x + 3) / 6

class h_swish(nn.Module):
    def __init__(self, inplace=True):
        super(h_swish, self).__init__()
        self.sigmoid = h_sigmoid(inplace=inplace)

    def forward(self, x):
        return x * self.sigmoid(x)
class CoordAtt(nn.Module):
    def __init__(self, inp, oup, reduction):
        super(CoordAtt, self).__init__()
        self.pool_h = nn.AdaptiveAvgPool2d((None, 1))
        self.pool_w = nn.AdaptiveAvgPool2d((1, None))

        mip = max(8, inp // reduction)

        self.conv1 = nn.Conv2d(inp, mip, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(mip)
        self.act = h_swish()

        self.conv_h = nn.Conv2d(mip, oup, kernel_size=1, stride=1, padding=0)
        self.conv_w = nn.Conv2d(mip, oup, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        identity = x

        n, c, h, w = x.size()
        x_h = self.pool_h(x)
        x_w = self.pool_w(x).permute(0, 1, 3, 2)

        y = torch.cat([x_h, x_w], dim=2)
        
        y = self.conv1(y)
        y = self.bn1(y)
        y = self.act(y)
    
        x_h, x_w = torch.split(y, [h, w], dim=2)
        x_w = x_w.permute(0, 1, 3, 2)
        
        a_h = self.conv_h(x_h).sigmoid()
        a_w = self.conv_w(x_w).sigmoid()
# 如果下面这个原论文代码用不了的话，可以换成另一个试试
        out = identity * a_w * a_h 
        # out = a_h.expand_as(x) * a_w.expand_as(x) * identity
        return out

In [3]:
class ResUnit(nn.Module):
    def __init__(self, in_channels, out_channels, lng, lat):
        # 模型的输入是一个四维张量 (B, C, lng, lat)
        super(ResUnit, self).__init__()
        self.bn1 = nn.BatchNorm2d(60)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 1, padding=1)
        self.bn2 = nn.BatchNorm2d(60)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, padding=1)
        self.conv3 = nn.Conv2d(out_channels, out_channels, 3, 1, 1)
        self.relu = nn.ReLU(inplace = True)
        self.CoordAtt = CoordAtt(60,60,4)        
        self.fc = nn.Linear(1024, 128)
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(1024, 128)
        self.fc0 = nn.Linear(128, 1024)
        # 分别定义两个批归一化层和卷积层
    def forward(self, x):
        z = F.relu(self.bn1(x))
        z = self.conv1(z)
        z = F.relu(self.bn2(z))
        z = self.conv2(z)
        z = z.reshape(-1,60,1024)
        Qs = self.fc(z)
        Ks = self.fc1(z).permute(0, 2, 1) 
        Vs = self.fc2(z)
        attention_scores = torch.matmul(Qs, Ks)
        attention_scores = attention_scores / math.sqrt(61440)
        attention_scores = nn.Softmax(dim=-1)(attention_scores)
        z = torch.matmul(attention_scores, Vs)
        z = self.fc0(z)
        z = z.reshape(-1,60,32,32)
        z = self.CoordAtt(z)
        out = x+z
        return out 

In [4]:
class Branch_net(nn.Module):
    def __init__(self, num_res_unit, input_lenght, flow_channel, grid_heigh, grid_width,in_channels):
        super(Branch_net, self).__init__()
        self.num_res_unit = num_res_unit
        self.input_lenght = input_lenght
        self.flow_channel = flow_channel
        self.grid_heigh = grid_heigh
        self.grid_width = grid_width
        self.in_channels = in_channels
        # 每个分支网络的首部都是用一个卷积网络将输入的特征维度从低维映射到高维
        self.branch_net = nn.ModuleList([ResUnit(60, 60, self.grid_heigh, self.grid_width)])
        # 接下来依次添加多个残差卷积单元
        for i in range(1): 
            self.branch_net.append(ResUnit(60, 60, self.grid_heigh, self.grid_width))
        self.branch_net.append(nn.Conv2d(60, 2, kernel_size=3,stride = 1, padding = 1))
    # 分支网络的前向传播
    def forward(self, x):
        for layer in self.branch_net:
             x = layer(x)
        return x 

In [5]:
class STEResUnit1(nn.Module):
    def __init__(self, in_channels, out_channels, i):
        super(STEResUnit1,self).__init__()
        self.i=i
        self.conv3d0  = nn.Conv3d(in_channels, out_channels, kernel_size=(1,3,3), dilation =(1,1,1), stride=1, padding='same')
        self.conv3d00  = nn.Conv3d(out_channels, out_channels, kernel_size=(3,1,1), dilation =(1,1,1), stride=1, padding='same')
        self.conv1 = nn.Conv3d(out_channels, 2, kernel_size=(1,1,1), dilation =(1,1,1),stride=1, padding='same')
    def forward(self,x):
        x0 = self.conv3d0(x)
        X = self.conv3d00(x0)
        if self.i ==True:
            X = self.conv1(X)
            return X
        else:
            return torch.cat((X,x),1)
class DenseNet(nn.Sequential):
    """DenseBlock"""
    def __init__(self, num_layers, num_input_features,num_output):
        super(DenseNet, self).__init__() 
        for i in range(num_layers):
            r = False
            if i ==num_layers-1:
                r = True
            layer = STEResUnit1(num_input_features+i*num_output, num_output,r)
            self.add_module("denselayer%d" % (i+1), layer)
    def forward(self, x):
        out = super(DenseNet, self).forward(x)
        return out

In [7]:
class Model(nn.Module):
    def __init__(self,lr=0.01,epoch=10, #训练轮次数
                 batch_size=32, #批训练batch大小
                 c_length=3, #邻近性时间流量特征序列默认长度
                 p_length=348654684896, #周期性时间流量特征序列默认长度
                 t_length=2,  #趋势性时间流量特征序列默认长度
                 external_dim=28, #外部特征的维度
                 grid_heigh=32, #网格图的高度
                 grid_width=32, #网格图的宽度
                 flow_channel=2, #流量种类
                 num_res_unit=2, #设定残差卷积单元的数量
                 in_channels = 6,
                 data_min = -10000,  # 输入数据的最小值默认值
                 data_max = 10000):
        super(Model, self).__init__()
        self.epoch = epoch  
        self.lr = lr
        self.batch_size = batch_size
        self.c_length = c_length
        self.p_length = p_length
        self.t_length = t_length
        self.external_dim = external_dim
        self.grid_heigh = grid_heigh
        self.grid_width = grid_width
        self.in_channels = in_channels
        self.flow_channel  = flow_channel 
        self.num_res_unit = num_res_unit
        self.logger = logging.getLogger(__name__)
        self.data_min = data_min
        self.data_max = data_max
        self.gpu_available = torch.cuda.is_available()
        if self.gpu_available:  #如果GPU存在，则调用
            self.gpu = torch.device("cuda:0")
        self.backbone_net()
        self.save_path="L%d_C%d_P%d_T%d/"% (self.num_res_unit,self.c_length, self.p_length, self.t_length)
        self.best_mse = 10000
    def backbone_net(self):
        self.c_net = Branch_net(self.num_res_unit,self.c_length,
                     self.flow_channel, self.grid_heigh, self.grid_width,60) 
        self.conv3D1 = nn.Conv3d(2, 8, kernel_size=(1,1,1), stride=1, padding='same')
        self.conv3D11 = nn.Conv3d(2, 8, kernel_size=(1,1,1), stride=1, padding='same')
        self.conv3D111 = nn.Conv3d(2, 8, kernel_size=(1,1,1), stride=1, padding='same')
        self.conv3D222 = nn.Conv3d(8, 2, kernel_size=(1,1,1), stride=1, padding='same')
        self.conv3D22 = nn.Conv3d(8, 2, kernel_size=(1,1,1), stride=1, padding='same')
        self.DenseNet = DenseNet(4,8,8)
        self.conv1 = nn.Conv2d(19, 60, 3, 1, 1)
        self.ext_net1 = nn.Sequential(
            nn.Linear(28, 15), 
            nn.ReLU(inplace = True),
            nn.Dropout(0.25),
            nn.Linear(15,1*self.grid_heigh*self.grid_width))
        self.lstm = LSTM(28,32,1,28)
        self.dropout = nn.Dropout(0.3)
    def forward(self, xc, xp, xt, ext,xw):
        ext = self.lstm(ext)
        ext = self.dropout(ext)
        ext = self.ext_net1(ext).view([-1, 1,
                                          self.grid_heigh, self.grid_width])
        xcin = xc[:,0:11:2,:,:] #clossness
        xcin = xcin.reshape(-1,1,6,32,32)
        xcout = xc[:,1:12:2,:,:]
        xcout = xcout.reshape(-1,1,6,32,32)
        Xc = torch.cat((xcin,xcout),1)
        
        xpin = xp[:,:5:2,:,:] #period
        xpin = xpin.reshape(-1,1,3,32,32)
        xpout = xp[:,1:6:2,:,:]
        xpout = xpout.reshape(-1,1,3,32,32)
        Xp = torch.cat((xpin,xpout),1)
        
        xtin = xt[:,:5:2,:,:] #trend
        xtin = xtin.reshape(-1,1,3,32,32)
        xtout = xt[:,1:6:2,:,:]
        xtout = xtout.reshape(-1,1,3,32,32)
        
        Xt = torch.cat((xtin,xtout),1)
        Xc = self.conv3D1(Xc)
        Xc = self.DenseNet(Xc)
        Xc = Xc.reshape(-1,12,32,32)
        Xp = self.conv3D11(Xp)
        Xp = self.conv3D22(Xp)
        Xp = Xp.reshape(-1,6,32,32)
        Xt = self.conv3D111(Xt)
        Xt = self.conv3D222(Xt)
        Xt = Xt.reshape(-1,6,32,32)

        XC = torch.cat((Xc,Xp,Xt,ext),1)
        XC = self.conv1(XC)
        XC = self.c_net(XC)
        XC = torch.tanh(XC)    
        return XC
    def train_model(self, train_loader, val_loader):
        optimizer = optim.Adam(self.parameters(),lr = self.lr)
        loss_func = nn.MSELoss() # 定义模型的优化器和损失函数
#         loss_func = nn.L1Loss(size_average=None, reduce=None, reduction='mean')
        early_stop_threshold = 10 # 设定提前停止阈值
        epoch_count = 0
        start_time = time.time()
        for ep in range(self.epoch):
            loss_list = []
            rmse_list = []
            mae_list = []
            self.train() # 启动模型训练
            for i, (xc, xp, xt, xe, xw,y) in enumerate(train_loader):
                if self.gpu_available:
                    xc = xc.to(self.gpu)
                    xp = xp.to(self.gpu)
                    xt = xt.to(self.gpu)
                    xe = xe.to(self.gpu)
                    xw = xw.to(self.gpu)
                    y = y.to(self.gpu)
                ypred = self.forward(xc, xp, xt, xe,xw)
                loss = loss_func(ypred, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            # 模型验证部分，根据验证集上的损失保存最优模型
            for i, (xc, xp, xt, xe,xw, y) in enumerate(val_loader):
                if self.gpu_available:
                    xc = xc.to(self.gpu)
                    xp = xp.to(self.gpu)
                    xt = xt.to(self.gpu)
                    xe = xe.to(self.gpu)
                    xw = xw.to(self.gpu)
                    y = y.to(self.gpu)
                ypred = self.forward(xc, xp, xt, xe,xw)
                val_loss = loss_func(ypred, y)
                loss_list.append(val_loss.item())
                end_time = time.time() 
            val_mse = np.mean(loss_list)
            print("[%.2fs] ep %d val mse %.6f" %(end_time - start_time, ep, val_mse))
            if val_mse < self.best_mse: #保存当前更优模型参数
                self.save_model("ST-D3DDARN")
                self.best_mse = val_mse
                epoch_count = 0
            else:
                epoch_count = epoch_count +1
#             if epoch_count >= early_stop_threshold:
#                 break  # 当超过提前停止阈值时整个循环结束
            self.eval()
            for i, (xc, xp, xt, xe,xw, y) in enumerate(test_loader):
                if self.gpu_available:
                    xc = xc.to(self.gpu)
                    xp = xp.to(self.gpu)
                    xt = xt.to(self.gpu)
                    xe = xe.to(self.gpu)
                    xw = xw.to(self.gpu)
                    y = y.to(self.gpu)
                
                with torch.no_grad():
                    ypred = self.forward(xc, xp, xt, xe,xw)
                # 采用RMSE和MAE作为模型评估的指标
#                 rmse = ((ypred - y) **2).mean().pow(1/2)
#                 mae = ((ypred - y).abs()).mean()
#                 # 将RMSE和MAE恢复到规范化之前的尺度
#                 rmse = rmse * (self.data_max - self.data_min)
#                 mae = mae * (self.data_max - self.data_min)
#                 rmse_list.append(rmse.item())
#                 mae_list.append(mae.item())
                ypred = ypred.cpu().numpy()
                y = y.cpu().numpy()
                if i == 0:
                    ypred1 = ypred
                    y1 = y
                else:
                    y1 = np.concatenate((y1,y),axis=0)
                    ypred1 = np.concatenate((ypred1,ypred),axis=0)
            y = [inverse_transform(d) for d in y1]
            ypred = [inverse_transform(d) for d in ypred1]
            y1 = np.array(y).reshape(2752512)
            ypred1 = np.array(ypred).reshape(2752512)
            print(np.sqrt(metrics.mean_squared_error(ypred1, y1)))
            print(metrics.mean_absolute_error(ypred1, y1))
            LISTRMSE.append(np.sqrt(metrics.mean_squared_error(ypred1, y1)))
            LISTMAE.append(metrics.mean_absolute_error(ypred1, y1))
    # 模型的评估测试集启动部分
    def test_model(self, test_loader):
        rmse_list = []
        mae_list = []
        self.eval() # 启动模型评估（固定BN层参数）
        for i, (xc, xp, xt, xe,xw, y) in enumerate(test_loader):
            if self.gpu_available:
                xc = xc.to(self.gpu)
                xp = xp.to(self.gpu)
                xt = xt.to(self.gpu)
                xe = xe.to(self.gpu)
                xw = xw.to(self.gpu)
                y = y.to(self.gpu)
             
            with torch.no_grad():
                ypred = self.forward(xc, xp, xt, xe,xw)
            # 采用RMSE和MAE作为模型评估的指标
            rmse = ((ypred - y) **2).mean().pow(1/2)
            mae = ((ypred - y).abs()).mean()
            # 将RMSE和MAE恢复到规范化之前的尺度
            rmse = rmse * (self.data_max - self.data_min)
            mae = mae * (self.data_max - self.data_min)
            rmse_list.append(rmse.item())
            mae_list.append(mae.item())
            ypred = ypred.cpu().numpy()
            y = y.cpu().numpy()
            if i == 0:
                ypred1 = ypred
                y1 = y
            else:
                y1 = np.concatenate((y1,y),axis=0)
                ypred1 = np.concatenate((ypred1,ypred),axis=0)
        
        mae_test = np.mean(mae_list)
        rmse_test = np.mean(rmse_list)
        return rmse_test, mae_test, ypred1, y1
    
    # 模型保存代码
    def save_model(self, name):
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)
        torch.save(self.state_dict(), self.save_path + name + ".pkl")
    
    # 读取已保存的模型
    def load_model(self, name):
        if not name.endswith(".pkl"):
            name += ".pkl"
        self.load_state_dict(torch.load(self.save_path + name)) 

In [8]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
gpu_available = torch.cuda.is_available()
print(torch.cuda.is_available())
if gpu_available:
    gpu = torch.device("cuda:0")
    print(1)

cuda
True
1


In [9]:
def inverse_transform(X):
    X = (X + 1.) / 2.
    X = 1. * X * (1250 - 0) + 0
    return X
for i in range(1):
    if __name__ == '__main__':
        logging.basicConfig(level=logging.DEBUG,format='%(levelname)s-%(message)s')
        # 为模型中所有超参数赋值
        epochs = 100  # 训练的轮次数
        batch_size = 32 # 训练批次大小
        T = 48  # 一天内的时间片段划分
        c_length = 6  # 临近性序列长度
        p_length = 1  # 周期性序列长度
        t_length = 1  # 趋势性序列长度
        grid_height, grid_width = 32, 32  # 网格图的规格
        flow_channel = 2 # 流量类别数
        lr = 0.0005  # 学习率
        external_dim = 28 #额外特征维度
        num_res_unit = 2  # 卷积残差单元个数

        # 读取训练,验证和测试数据（分别需读入临近性，周期性，趋势性，额外特征数据已经标签数据）
        c_train = np.load('data/c_train.npy')
        p_train = np.load('data/p_train.npy')
        t_train = np.load('data/t_train.npy')
        e_train = np.load('data/e_train.npy')
        w_train = np.load('data/w_train.npy')
        train_y = np.load('data/train_y.npy')
        c_val = np.load('data/c_val.npy')
        p_val = np.load('data/p_val.npy')
        t_val = np.load('data/t_val.npy')
        e_val = np.load('data/e_val.npy')
        w_val = np.load('data/w_val.npy')
        val_y = np.load('data/val_y.npy')
        c_test = np.load('data/c_test.npy')
        p_test = np.load('data/p_test.npy')
        t_test = np.load('data/t_test.npy')
        e_test = np.load('data/e_test.npy')
        w_test = np.load('data/w_test.npy')
        test_y = np.load('data/test_y.npy')

        # 读取已保存的归一化数据转换器
        with open ('../data_process/data/preprocessing.pkl', 'rb') as f:
         scale = pickle.load(f)  
        # 将训练集/验证集/测试集都整理成pytorch的输入格式
        train_set = TensorDataset(torch.Tensor(c_train),torch.Tensor(p_train),torch.Tensor(t_train), 
                torch.Tensor(e_train),torch.Tensor(w_train), torch.Tensor(train_y))
        val_set = TensorDataset(torch.Tensor(c_val),torch.Tensor(p_val),torch.Tensor(t_val),
               torch.Tensor(e_val),torch.Tensor(w_val),  torch.Tensor(val_y))
        test_set = TensorDataset(torch.Tensor(c_test),torch.Tensor(p_test),torch.Tensor(t_test),
               torch.Tensor(e_test), torch.Tensor(w_test), torch.Tensor(test_y))

        # 装载训练集/验证集/测试集数据
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle = True,drop_last=False)
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle = False,drop_last = False)
        test_loader = DataLoader(test_set, batch_size=batch_size, shuffle = False,drop_last = False)
        # 定义模型
        net = Model(lr = lr, epoch = epochs, batch_size = batch_size, 
                   c_length = c_length, p_length = p_length, t_length = t_length, 
                   external_dim = external_dim, grid_heigh = grid_height, 
                   grid_width = grid_width, flow_channel = flow_channel, 
                   num_res_unit = num_res_unit, data_min = scale._min, data_max = scale._max)

        if gpu_available:
            net = net.to(gpu)
        # 训练模型
        net.train_model(train_loader, val_loader)
        print('test model....')
        net.load_model("ST-D3DDARN") # 读取最优模型
        rmse, mae,ypred,y  = net.test_model(test_loader) #用验证集上最好的模型进行测试评估
        y = [inverse_transform(d) for d in y]
        ypred = [inverse_transform(d) for d in ypred]
        y1 = np.array(y).reshape(2752512)
        ypred1 = np.array(ypred).reshape(2752512)
        print('RMSE', np.sqrt(metrics.mean_squared_error(ypred1, y1)))
        print('MAE', metrics.mean_absolute_error(ypred1, y1))

[14.12s] ep 0 val mse 0.004042
39.851147
23.06564
[26.71s] ep 1 val mse 0.003146
35.1136
19.98656
[39.01s] ep 2 val mse 0.002460
29.558651
16.640697
[51.43s] ep 3 val mse 0.002818
30.544802
17.098852
[64.34s] ep 4 val mse 0.002294
26.902086
15.62105
[77.25s] ep 5 val mse 0.002125
25.780987
14.556032
[90.03s] ep 6 val mse 0.002040
25.347193
14.401421
[103.20s] ep 7 val mse 0.001906
24.817684
13.762437
[116.45s] ep 8 val mse 0.001862
25.702
14.572268
[129.24s] ep 9 val mse 0.001835
24.429092
13.79668
[142.20s] ep 10 val mse 0.002397
27.038414
15.411413
[154.64s] ep 11 val mse 0.001717
23.239386
12.853266
[167.34s] ep 12 val mse 0.001610
22.788239
12.740134
[180.30s] ep 13 val mse 0.001722
23.840511
13.512018
[193.16s] ep 14 val mse 0.001620
22.398159
12.353111
[205.58s] ep 15 val mse 0.001993
26.776758
15.597865
[218.23s] ep 16 val mse 0.001530
22.520975
12.362679
[230.74s] ep 17 val mse 0.001753
25.850632
14.282771
[243.01s] ep 18 val mse 0.001410
21.389608
11.969367
[255.25s] ep 19 val